# Langfuseを使用したDatabricksモデルの可観測性

Databricksは、大規模な言語モデルをホスティングおよび提供するための強力なプラットフォームを提供します。Databricksの提供エンドポイントと**Langfuse**を組み合わせることで、開発および本番環境でAIワークロードをトレース、監視、および分析できます。

このノートブックでは、Langfuseを使用してDatabricksモデルを使用する**3つ**の異なる方法を示します：
1. **OpenAI SDK:** OpenAI SDKを介してDatabricksモデルエンドポイントを使用します。
2. **LangChain:** LangChainパイプラインでDatabricks LLMインターフェースと統合します。
3. **LlamaIndex:** LlamaIndex内でDatabricksエンドポイントを使用します。

> **Databricks Model Servingとは？**  
Databricks Model Servingは、大規模なモデルを本番環境で提供するための自動スケーリングと堅牢なインフラストラクチャを備えています。また、プライベートデータでLLMを微調整することも可能で、データプライバシーを維持しながら独自の情報を活用できます。

> **Langfuseとは？**  
[Langfuse](https://langfuse.com)は、LLMの可観測性と監視のためのオープンソースプラットフォームです。メタデータ、プロンプトの詳細、トークン使用量、レイテンシーなどをキャプチャすることで、AIアプリケーションをトレースおよび監視するのに役立ちます。

## 1. 依存関係のインストール

始める前に、Python環境に必要なパッケージをインストールします：

- **openai**: OpenAI SDKを介してDatabricksエンドポイントを呼び出すために必要です。
- **databricks-langchain**: "OpenAIのような"インターフェースを介してDatabricksエンドポイントを呼び出すために必要です。
- **llama-index** および **llama-index-llms-databricks**: LlamaIndex内でDatabricksエンドポイントを使用するために必要です。
- **langfuse**: Langfuseプラットフォームにトレースデータを送信するために必要です。

In [0]:
%pip install openai langfuse databricks-langchain llama-index llama-index-llms-databricks
%restart_python

## 2. 環境変数の設定

**Langfuse**のクレデンシャルと**Databricks**のクレデンシャルを環境変数として設定します。以下のダミーキーをそれぞれのアカウントから取得した実際のキーに置き換えてください。

 - `LANGFUSE_PUBLIC_KEY` / `LANGFUSE_SECRET_KEY`: Langfuseプロジェクト設定から取得します。
 - `LANGFUSE_HOST`: `https://cloud.langfuse.com`（EUリージョン）または `https://us.cloud.langfuse.com`（USリージョン）。
 - `DATABRICKS_TOKEN`: Databricksのパーソナルアクセストークン。
 - `DATABRICKS_HOST`: DatabricksワークスペースのURL（例：`https://dbc-xxxxxxx.cloud.databricks.com`）。

In [0]:
import os

# 環境変数の例（実際のキー/トークンに置き換えてください）
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..."  # あなたのパブリックキー
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..."  # あなたのシークレットキー
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"  # または https://us.cloud.langfuse.com

os.environ["DATABRICKS_TOKEN"] = "dapi..."  # Databricks パーソナルアクセストークン
os.environ["DATABRICKS_HOST"] = "https://....cloud.databricks.com/"

## アプローチ1: OpenAI SDKを介したDatabricksモデルの使用

Databricksエンドポイントは、OpenAI APIの代替として機能できます。これにより、`openai`ライブラリに依存する既存のコードと簡単に統合できます。内部では、`langfuse.openai.OpenAI`がリクエストを自動的にLangfuseにトレースします。

### 手順
1. `langfuse.openai`から`OpenAI`クライアントをインポートします。
2. クライアントを作成し、`api_key`にDatabricksトークンを、`base_url`にDatabricksワークスペースエンドポイントを設定します。
3. クライアントの`chat.completions.create()`メソッドを使用してプロンプトを送信します。
4. Langfuseダッシュボードでトレースを確認します。

**注:** Langfuseを使用したOpenAIのトレースに関する詳細な例については、[OpenAI統合ドキュメント](https://langfuse.com/docs/integrations/openai/python/get-started)を参照してください。

In [0]:
# Langfuse OpenAI クライアント
from langfuse.openai import OpenAI

# 環境変数を取得
databricks_token = os.environ.get("DATABRICKS_TOKEN")
databricks_host = os.environ.get("DATABRICKS_HOST")

# Databricks を指す OpenAI ライクなクライアントを作成
client = OpenAI(
    api_key=databricks_token,  # Databricks パーソナルアクセストークン
    base_url=f"{databricks_host}/serving-endpoints",  # あなたの Databricks ワークスペース
)

In [0]:
response = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "あなたはAIアシスタントです。"},
        {"role": "user", "content": "Databricksとは何ですか？"}
    ],
    model="databricks-meta-llama-3-3-70b-instruct",  # Databricksモデルサービングエンドポイント名に基づいて調整
    max_tokens=256
)

# モデルからの応答を表示
display(response.choices[0].message.content)

リクエストが完了したら、**Langfuseダッシュボードにログイン**して新しいトレースを探します。プロンプト、応答、レイテンシ、トークン使用量などの詳細が表示されます。

![LangfuseのDatabricks例のトレース](https://raw.githubusercontent.com/images/docs/databricks/databricks-example-trace-openai-sdk.png)

_[Langfuseの公開トレースへのリンク](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/029b2344-e2a2-4c52-8d04-bd71f70c1120?timestamp=2025-03-06T14%3A45%3A04.141Z)_

## アプローチ2: LangChainの使用

DatabricksモデルはLangChainを介しても使用できます。[`ChatDatabricks`](https://python.langchain.com/docs/integrations/chat/databricks/)クラスは、Databricksのモデルサービングエンドポイントをラップします。

### 手順
1. 環境変数として`DATABRICKS_HOST`を設定します。
2. トレースデータを自動的に収集するLangfuseの`CallbackHandler`を初期化します。
3. エンドポイント名、温度、その他のパラメータを使用して`ChatDatabricks`を使用します。
4. メッセージでモデルを呼び出し、Langfuseのコールバックハンドラーを渡します。
5. Langfuseダッシュボードでトレースを確認します。

**注:** Langfuseを使用したLangChainのトレースに関する詳細な例については、[LangChain統合ドキュメント](https://langfuse.com/docs/integrations/langchain/tracing)を参照してください。

In [0]:
from langfuse.callback import CallbackHandler

# Langfuseコールバックハンドラーを初期化
langfuse_handler = CallbackHandler(
    secret_key=os.environ.get("LANGFUSE_SECRET_KEY"),
    public_key=os.environ.get("LANGFUSE_PUBLIC_KEY"),
    host=os.environ.get("LANGFUSE_HOST")
)

In [0]:
from databricks_langchain import ChatDatabricks

chat_model = ChatDatabricks(
    endpoint="databricks-meta-llama-3-3-70b-instruct",   # Databricksモデルサービングエンドポイント名
    temperature=0.1,
    max_tokens=256,
    # 他のパラメータをここに追加できます
)

# プロンプトをシステム/ユーザーメッセージのリストとして構築
messages = [
    ("system", "あなたはDatabricksに関する質問に答えることができるチャットボットです。"),
    ("user", "Databricks Model Servingとは何ですか？")
]

# LangChainの.invoke()メソッドを使用してモデルを呼び出す
chat_model.invoke(messages, config={"callbacks": [langfuse_handler]})

コードを実行した後、Langfuseダッシュボードを開いて記録された会話を確認します。

![LangfuseのDatabricks例のトレース](https://langfuse.com/images/docs/databricks/databricks-example-trace-langchain.png)

_[Langfuseの公開トレースへのリンク](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/a55411bb-4bb4-435c-b922-e446683888ff?timestamp=2025-03-06T14%3A57%3A59.273Z)_

## アプローチ3: LlamaIndexの使用

データの取り込み、インデックス作成、またはRAGに[LlamaIndex](https://github.com/jerryjliu/llama_index)を使用する場合、デフォルトのLLMをDatabricksエンドポイントに置き換えることができます。

### 手順
1. `llama_index.llms.databricks`から`Databricks`をインポートします。
2. エンドポイント名とDatabricksの認証情報を使用して`Databricks` LLMを初期化します。
3. `langfuse.llama_index`から`LlamaIndexInstrumentor`を使用して自動トレースを有効にします。
4. チャットリクエストでLLMを呼び出します。
5. Langfuseダッシュボードでトレースを確認します。

**注:** Langfuseを使用したLlamaIndexのトレースに関する詳細な例については、[LlamaIndex統合ドキュメント](https://langfuse.com/docs/integrations/llama-index/get-started)を参照してください。

In [0]:
from llama_index.llms.databricks import Databricks

# Databricks LLMインスタンスを作成
llm = Databricks(
    model="databricks-meta-llama-3-3-70b-instruct",  # Databricksモデルサービングエンドポイント名
    api_key=os.environ.get("DATABRICKS_TOKEN"),
    api_base=f"{os.environ.get('DATABRICKS_HOST')}/serving-endpoints/"
)

In [0]:
from langfuse.llama_index import LlamaIndexInstrumentor
from llama_index.core.llms import ChatMessage

# LlamaIndexの操作をトレースするためにLlamaIndexInstrumentorを初期化
instrumentor = LlamaIndexInstrumentor(
    secret_key=os.environ.get("LANGFUSE_SECRET_KEY"),
    public_key=os.environ.get("LANGFUSE_PUBLIC_KEY"),
    host=os.environ.get("LANGFUSE_HOST")
)

# 自動トレースを開始
instrumentor.start()

In [0]:
messages = [
    ChatMessage(role="system", content="あなたは役に立つアシスタントです。"),
    ChatMessage(role="user", content="Databricksとは何ですか？")
]

response = llm.chat(messages)
display(response)

# Flush any pending events to Langfuse
instrumentor.flush()

LlamaIndexの呼び出しを確認するためにLangfuseにログインし、プロンプト、トークン使用量、完了データなどの詳細を確認できます。

![LangfuseでのDatabricksのLlamaIndexトレース例](https://langfuse.com/images/docs/databricks/databricks-example-llamaindex-trace.png)

_[Langfuseの公開トレースへのリンク](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/589a858e-9678-4624-bcb6-2e0266ecb1b3?timestamp=2025-03-06T15%3A10%3A02.467Z&observation=dd7b6235-6c92-4c9b-a966-872bc281c060)_

## トレースの強化 (オプション)

Langfuseは、より豊富なトレースデータのための追加機能をサポートしています:

- トレースに[メタデータ](https://langfuse.com/docs/tracing-features/metadata)、[タグ](https://langfuse.com/docs/tracing-features/tags)、[ログレベル](https://langfuse.com/docs/tracing-features/log-levels)、および[ユーザーID](https://langfuse.com/docs/tracing-features/users)を追加
- トレースを[セッション](https://langfuse.com/docs/tracing-features/sessions)でグループ化
- 追加のアプリケーションロジックをトレースするための[`@observe()`デコレータ](https://langfuse.com/docs/sdk/python/decorators)
- [Langfuseプロンプト管理](https://langfuse.com/docs/prompts/get-started)を使用し、プロンプトをトレースにリンク
- トレースに[スコア](https://langfuse.com/docs/scores/custom)を追加

詳細については、[Langfuseのドキュメント](https://langfuse.com/docs)を参照してください。

## 次のステップ
- Langfuse PlaygroundでDatabricksモデルを使用する方法やLLM-as-a-Judge評価については[こちら](https://langfuse.com/docs/integrations/databricks/use-with-playground-and-evals.mdx)をご覧ください。
- 高度なモデルサービング構成については[Databricksのドキュメント](https://docs.databricks.com/aws/ja/machine-learning/model-serving/manage-serving-endpoints)を参照してください。
- アプリケーション全体のフローを追跡するための[Langfuseトレース機能](https://langfuse.com/docs)について学びましょう。
- Langfuseの[プロンプト管理](https://langfuse.com/docs/prompts/get-started)を試すか、[LLM-as-a-Judge評価](https://langfuse.com/docs/scores/model-based-evals)を設定してみましょう。